In [1]:
import nltk.classify.util
from nltk.corpus import stopwords
import string
import json
import re
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import sentiment_analyser as s
import sys
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import pandas as pd
import time, sys
import csv

record = []

### Emojis detection

In [2]:

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

### Regular expression for data cleaning

In [3]:

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', 
    r'(?:\S)'
]

In [4]:
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

### Word tokenization

In [5]:
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

### Converting tweets to Dataframe

In [6]:
df = pd.DataFrame()
cols = ["Tweet", "Time", "User_Location", "Sentiment", "Retweets"]

### Live streaming of Depressive Tweets

In [7]:
class listener(StreamListener):
    
    def __init__(self, time_limit=20):
        self.start_time = time.time()
        self.limit = time_limit
        self.saveFile = open('abcd.json', 'a')
        super(StreamListener, self).__init__()
        
    def on_data(self, data):        
        if (time.time() - self.start_time) < self.limit:
            try:
                all_data = json.loads(data)
                all_tweets = all_data["text"]
                user_location = all_data["user"]["location"] 
                created_at    = all_data["created_at"]
                retweeted     = all_data["retweeted"]
                
                analysis  = TextBlob(all_tweets)
                tweet_polarity = analysis.sentiment.polarity
                
                sentiment = 0
                if(tweet_polarity>0):
                    sentiment = 1
                else:
                    sentiment = 0

                all_tweets = preprocess(all_tweets)

                punctuation = list(string.punctuation)
                stop_words = set(stopwords.words('english') + punctuation + ['RT'])
                tweet = [tweets for tweets in all_tweets if tweets not in stop_words]                

                tweet = ' '.join(str(e) for e in tweet)

                sentiment_value, confidence = s.sentiment(tweet)

                print(tweet, user_location, created_at, sentiment_value, retweeted, confidence)
                if len(tweet)>3:
                    record.append([tweet, created_at, user_location, sentiment_value, retweeted])                

                if confidence*100 >= 80:
                    output = open("twitter-out.txt","a")
                    output.write(tweet)
                    output.write("\t")
                    output.write(sentiment_value)
                    output.write('\n')
                    output.close()

                return True        
            
            except BaseException as e:                
                return True
            return False
        else:
            print("STOPPED!!!!!!!!!!!!!!!!!!!!!!")
            return False

    def on_error(self, status):
        print(status)

### Tag to scrape Depressive tweets

In [8]:
tags = input('Enter the tag you want sentiment review of:')

Enter the tag you want sentiment review of:depression


### Connecting to the Twitter API

In [9]:
ckey="BGvTRmO9b9ZySs8hY8n0WjcXy"
csecret="Ki3lHvwLZZrG5zZFU7RtknhDlJgi5zDt0eVDNiPWHc3PJmdLj9"
atoken="1382391655223873539-2lvJk0qm5gxcBQyppOB9bp7DpSDsPK"
asecret="LjvoNphGxPGNOsnWR34hJO3YUtweZ7VjBabAPujhtDdSv"

In [10]:
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

In [11]:
twitterStream = Stream(auth, listener())
twitterStream.filter(track=[tags], languages=["en"])

STOPPED!!!!!!!!!!!!!!!!!!!!!!


### Storing tweets in a CSV file 

In [12]:
df = pd.DataFrame(record, columns=cols)
df.to_csv('tweetss.csv', sep=',', index=False)
df.head()

,Tweet,Time,User_Location,Sentiment,Retweets


Code scrapes data dynamically from the Twitter API page